Для начала настроим маршрут для редактирования записей. Форму для редактирования будем возвращать при GET-запросах к адресам вида *birthday/<pk>/edit/*. 

Запросы на редактирование будем обрабатывать в той же view-функции `birthday()`.

Добавьте новую строчку в файл *birthday/urls.py*:

```py
# birthday/urls.py
from django.urls import path

from . import views

app_name = 'birthday'

urlpatterns = [
    path('', views.birthday, name='create'),
    path('list/', views.birthday_list, name='list'),
    path('<int:pk>/edit/', views.birthday, name='edit'),
] 
```

Чтобы каждый раз не писать адрес страницы редактирования вручную — в шаблон страницы *birthday/list/* к каждой записи добавим ссылку на страницу редактирования. 



```html
<!-- templates/birthday/birthday_list.html -->
{% extends "base.html" %}

{% block content %}
  {% for birthday in birthdays %}
    <div>
      <div>
        {{ birthday.first_name }} {{ birthday.last_name }} - {{ birthday.birthday }}
      </div>
      <!-- Ссылку тоже обернём в div; адрес ссылки
        указываем через namespace:name -->
      <div>
        <a href="{% url 'birthday:edit' birthday.id %}">Изменить запись</a>
      </div>
      {% if not forloop.last %}
        <hr>
      {% endif %}
    </div>
  {% endfor %}
{% endblock %} 
```

С такими ссылками работать будет удобнее:

![alt text](https://pictures.s3.yandex.net/resources/162_1682598844.png)


Чтобы при отрисовке формы на странице в ней отображались значения полей объекта модели — надо передать объект модели в форму, в аргумент `instance`:

```py
# Находим запрошенный объект для редактирования по первичному ключу
# или возвращаем 404 ошибку, если такого объекта нет.
instance = get_object_or_404(Birthday, pk=pk)  
# Связываем форму с найденным объектом: передаём его в аргумент instance.
form = BirthdayForm(request.POST or None, instance=instance) 
```

С `form.instance` мы уже встречались при создании записи, но в том случае объект модели автоматически передавался в форму после сохранения нового объекта в БД. 

А для редактирования мы получаем объект модели с помощью запроса Django ORM — и передавать этот объект в форму нужно явным образом.

Такая возможность есть только у наследников `forms.ModelForm`, обычные `forms.Form` не могут быть привязаны к модели, а значит, объект формы нельзя связать с объектом модели.

Для обработки запросов на редактирование можно создать отдельную view-функцию, например, такую:

```py
# Импортируем шорткат для получения объекта или вызова 404 ошибки.
from django.shortcuts import get_object_or_404, render

from .forms import BirthdayForm
from .models import Birthday
from .utils import calculate_birthday_countdown


def edit_birthday(request, pk):
    # Находим запрошенный объект для редактирования по первичному ключу
    # или возвращаем 404 ошибку, если такого объекта нет.
    instance = get_object_or_404(Birthday, pk=pk)
    # Связываем форму с найденным объектом: передаём его в аргумент instance.
    form = BirthdayForm(request.POST or None, instance=instance)
    # Всё остальное без изменений.
    context = {'form': form}
    # Сохраняем данные, полученные из формы, и отправляем ответ:
    if form.is_valid():
        form.save()
        birthday_countdown = calculate_birthday_countdown(
            form.cleaned_data['birthday']
        )
        context.update({'birthday_countdown': birthday_countdown})
    return render(request, 'birthday/birthday.html', context)


def birthday(request):
    ... 
```

При этом варианте во view-функциях `edit_birthday()` и `birthday()` будет много повторяющегося кода. 

Лучше объединить создание и редактирование объекта в одной функции. 

```py
from django.shortcuts import get_object_or_404, render

from .forms import BirthdayForm
from .models import Birthday
from .utils import calculate_birthday_countdown


# Добавим опциональный параметр pk.
def birthday(request, pk=None):
    # Если в запросе указан pk (если получен запрос на редактирование объекта):
    if pk is not None:
        # Получаем объект модели или выбрасываем 404 ошибку.
        instance = get_object_or_404(Birthday, pk=pk)
    # Если в запросе не указан pk
    # (если получен запрос к странице создания записи):
    else:
        # Связывать форму с объектом не нужно, установим значение None.
        instance = None
    # Передаём в форму либо данные из запроса, либо None. 
    # В случае редактирования прикрепляем объект модели.
    form = BirthdayForm(request.POST or None, instance=instance)
    # Остальной код без изменений.
    context = {'form': form}
    # Сохраняем данные, полученные из формы, и отправляем ответ:
    if form.is_valid():
        form.save()
        birthday_countdown = calculate_birthday_countdown(
            form.cleaned_data['birthday']
        )
        context.update({'birthday_countdown': birthday_countdown})
    return render(request, 'birthday/birthday.html', context) 
```

В форме есть проблема: не загрузилась дата дня рождения. В базе данных это значение есть, но там оно хранится в формате ГГГГ-ММ-ДД, однако виджет Django ожидает дату в русской локализации: ДД.ММ.ГГГГ; виджет получает дату в другом формате и не может отобразить её. 

Есть простой способ исправить ситуацию: нужно отключить локализацию форматов (она включена по умолчанию), установив в settings.py значение `False` для ключа `USE_L10N`. Теперь Django будет работать с «английским» форматом даты — и виджет с удовольствием отобразит значение, полученное из базы.

Можно поправить формат и без изменения локализации: для этого нужно переопределить формат даты на уровне виджета, используя параметр `input_formats=['%Y-%m-%d']`.

Чтобы главная страница и страница редактирования внешне отличались друг от друга, добавим в шаблон *birthday/birthday.html* два варианта заголовка:

```html
<!-- birthday/birthday.html -->
...
{% block content %}
  <!-- Если в пути запроса содержится строка /edit/. -->
  {% if '/edit/' in request.path %}
    <!-- Вывести заголовок и номер записи. -->
    <h1>Редактировать запись {{ form.instance.pk }}</h1>
  {% else %}
    <h1>Создать запись</h1>
  {% endif %}
  <div class="card col-4 m-3">
    <div class="card-header">
      Калькулятор ожидания дня рождения
    </div>
    ... 
```

Теперь страница редактирования записи выглядит так:

![alt text](https://pictures.s3.yandex.net/resources/165_1682598921.png)


В заголовке «Редактировать запись» `id` записи мы получаем из объекта `form.instance` — это объект модели, связанный с формой. 

Чуть ниже в коде, в теге `{% with … %}`, этот объект передаётся в переменную `data`. Немного упростим код: перенесём тег `with` повыше — и тогда в заголовке можно будет обращаться к `data`, а не к `form.instance`.

```html
{% block content %}
  <!-- Переносим тег with в самое начало блока content. -->
  {% with data=form.instance %}
    {% if '/edit/' in request.path %}
      <!-- Вместо form.instance подставляем data. -->
      <h1>Редактировать запись {{ data.pk }}</h1>
    {% else %}
      <h1>Создать запись</h1>
    {% endif %}
    <div class="card col-4">
      <div class="card-header">
        Калькулятор ожидания дня рождения
      </div>
      <div class="card-body">
        <form method="post">
          {% csrf_token %}
          {% bootstrap_form form %}
          {% bootstrap_button button_type="submit" content="Отправить" %}
        </form>
      </div>
    </div>
    {% if form.is_valid %}
      ID записи в БД: {{ data.id }}
      <hr>
      <h2>Привет, {{ data.first_name }} {{ data.last_name }}</h2>
      {% if birthday_countdown == 0 %}
        <p>С днём рождения!</p>
      {% else %}
        <p>Осталось дней до дня рождения: {{ birthday_countdown }}!</p>
      {% endif %}
    {% endif %}
  {% endwith %}
{% endblock %}
```

***
## Удаление объекта

Для удаления записей БД тоже нужна форма:

1. Удаление объекта — это операция, изменяющая состояние БД, следовательно, удалять объекты нужно POST-запросом. Отправить POST-запрос из браузера можно только из формы.

2. Удаление объекта — необратимое действие; лучше предоставить пользователю такой интерфейс, который даст возможность ещё раз обдумать и подтвердить свои действия.

Сейчас всё сделаем.

* Добавим маршрут для страницы удаления объекта.

* Создадим view-функцию для удаления объектов.

* Настроим автоматическую переадресацию: после удаления объекта пользователя надо перенаправить на какую-нибудь другую страницу, ведь страница, где находился пользователь, станет недоступна.

Измените код в *birthday/urls.py*: добавьте новый путь:

```py
# birthday/urls.py
from django.urls import path

from . import views

app_name = 'birthday'

urlpatterns = [
    path('', views.birthday, name='create'),
    path('list/', views.birthday_list, name='list'),
    path('<int:pk>/edit/', views.birthday, name='edit'),
    path('<int:pk>/delete/', views.delete_birthday, name='delete'),
] 
```

В файл *birthday/views.py* добавьте view-функцию `delete_birthday()`.

В файл birthday/views.py добавьте view-функцию `delete_birthday()`.
Перенесите код в свой проект.

```py
# birthday/views.py
# Дополнительно импортируйте шорткат для редиректа.
from django.shortcuts import get_object_or_404, redirect, render

...


def delete_birthday(request, pk):
    # Получаем объект модели или выбрасываем 404 ошибку.
    instance = get_object_or_404(Birthday, pk=pk)
    # В форму передаём только объект модели;
    # передавать в форму параметры запроса не нужно.
    form = BirthdayForm(instance=instance)
    context = {'form': form}
    # Если был получен POST-запрос...
    if request.method == 'POST':
        # ...удаляем объект:
        instance.delete()
        # ...и переадресовываем пользователя на страницу со списком записей.
        return redirect('birthday:list')
    # Если был получен GET-запрос — отображаем форму.
    return render(request, 'birthday/birthday.html', context)


...
```


1. При удалении объекта не нужно передавать в форму параметры запроса, достаточно связать форму и объект модели.

2. Не нужно проверять валидность формы, достаточно проверить только тип запроса.

3. Удаление объекта выполняем ORM-методом `delete()`.

4. После удаления объекта нужно перенаправить пользователя на страницу со списком записей. Для перенаправления применяется шорткат `redirect()`, в нём должно быть указано имя маршрута (`namespace:name`).

Немного дополним шаблон, чтобы заголовок страницы соответствовал действию:

```html
{% block content %}
  {% with data=form.instance %}
    {% if "/edit/" in request.path %}
      <h1>Редактировать запись {{ data.pk }}</h1>
    {% elif "/delete/" in request.path %}
      <h1>Удалить запись {{ data.pk }}</h1>
    {% else %}
      <h1>Создать запись</h1>
    {% endif %} 
```
Однако отображение формы может ввести пользователя в недоумение: он решит, что ему предлагают что-то отредактировать. Лучше бы не путать его и не давать возможность редактирования. 

Задача выглядит так: 

* нужно скрыть форму, чтобы не провоцировать пользователя на редактирование и возможные ошибки;

* нужно показать пользователю содержимое полей той записи, которую он удаляет (чтобы он убедился, что удаляет именно то, что планировал).

Все эти изменения можно внести прямо в шаблон *birthday/birthday.html*:

* если путь не содержит строку */delete/* — показываем форму;

* в ином случае (при запросе к адресу */delete/*) форму не показываем, а на страницу выводим значения полей объекта `form.instance`.

Измените код шаблона так, чтобы страница удаления объекта приняла примерно такой вид:

![alt text](https://pictures.s3.yandex.net/resources/168_1682599011.png)


```html
...
    <div class="card-body">
      <form method="post">
        {% csrf_token %}
        {% if not "/delete/" in request.path %}
          {% bootstrap_form form %}
        {% else %}
          <p>{{ data.first_name }} {{ data.last_name }}</p>
          <p>{{ data.birthday }}</p>
        {% endif %}
        {% bootstrap_button button_type="submit" content="Отправить" %}
      </form>
    </div>
...
```